In [1]:
import pybamm

In [21]:
# could implement a way to search what parameter you want to edit.
parameter_values.search("voltage")


Lower voltage cut-off [V]	2.0
Open-circuit voltage at 0% SOC [V]	2.0
Open-circuit voltage at 100% SOC [V]	3.65
Upper voltage cut-off [V]	3.65


In [37]:
# adjusting
parameter_values["Current function [A]"] = 10
parameter_values["Open-circuit voltage at 100% SOC [V]"] = 4.2

In [38]:
# Create a DFN model
model = pybamm.lithium_ion.DFN()

In [39]:
# solver
solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)

# Prevent solver failure if interpolant bounds are exceeded by a negligible amount
solver._on_extrapolation = "warn"

In [40]:
# Define an experiment
experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/5 for 10 hours or until 2.5 V",
            "Rest for 1 hour",
            "Charge at 1 A until 3.5 V",
            "Hold at 3.5 V until 10 mA",
            "Rest for 1 hour",
        ),
    ]
    * 2
)

In [41]:
# Create a simulation object
sim = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver, experiment=experiment)

In [42]:
# Simulate
sim.solve()

At t = 238.258 and h = 5.10856e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 238.258 and h = 1.11737e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 118.258 and h = 7.54588e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 233.714 and h = 2.1527e-07, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 236.506 and h = 1.21426e-09, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 135.305 and h = 1.82393e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 135.305 and h = 1.0935e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 257.867 and h = 3.58477e-09, the corrector convergence failed repeatedly or with |h| = hmin.


In [43]:
# Plot results
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=19.093923303580432, step=0.19093923303580432…

In [75]:
NMC=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\NMC\AE_gen1_BPX.json")
# LFP=(r"C:\Users\MVeerasi.ANALOG\Documents\InternshipFY24\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\LFP\lfp_18650_cell_BPX.json")
LFP=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json")

# OOP SUCKS

In [91]:
import pybamm

class SetupSimulation:
    def __init__(self, chemistry, electrochemical_model, solver) -> None:
        self.chemistry = chemistry
        self.adjusted_parameters = {} # Placeholder dictionary for user-adjusted parameters.
        self.electrochemical_model = electrochemical_model # electrochemistry-based lithium-ion battery model 
        self.solver = solver 
        # could put cell_type to help filter dupe chemistries?

    ADJUSTABLE_PARAMETERS = [
            "Ambient temperature [K]",
            "Initial temperature [K]",
            "Lower voltage cut-off [V]",
            "Upper voltage cut-off [V]",
            "Current function [A]",
        ]
    
    # This would be something like localhost:8080/chemistry=LFP/
    def set_battery_chemistry(self, chemistry):
        if chemistry == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
            # parameter_values = pybamm.ParameterValues("Chen2020")
            return parameter_values
        elif chemistry == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
            # parameter_values = pybamm.ParameterValues("Chen2020")
            return parameter_values
        
    # This would be something like localhost:8080/chemistry=LFP/model=dfn
    def set_electrochemical_model(self, electrochemical_model="DFN"): # default will be DFN, idk what it does and I will worry about this when it may matter - maybe
        if electrochemical_model == "DFN":
            return pybamm.lithium_ion.DFN()
        elif electrochemical_model == "SPM":
            return pybamm.lithium_ion.SPM()
        elif electrochemical_model == "SPMe":
            return pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Unknown model type: {electrochemical_model} Please pick DFN, SPM or SPMe")
        
    # # This would be something like localhost:8080/chemistry=LFP/model=dfn/adjustedParameters=true
    # def set_adjusted_parameter_values(self, adjusted_parameters):
    #     parameter_values.update(adjusted_parameters)
    
    # # This would be something like localhost:8080/chemistry=LFP/model=dfn/adjustedParameters=true/solver=casadi
    def set_solver(self, solver):
        solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)
        # Prevent solver failure if interpolant bounds are exceeded by a negligible amount
        solver._on_extrapolation = "warn"
        return solver

# from SetupSimulation import SetupSimulation - not needed in notebook
class SimulateBattery:
    def __init__(self, chemistry, electrochemical_model, solver):
        self.chemistry = chemistry
        self.electrochemical_model = electrochemical_model
        self.solver = solver
        self.setup_simulation = SetupSimulation(chemistry, electrochemical_model, solver)

    def simulate_battery_base_model(self):
        parameter_values = self.setup_simulation.set_battery_chemistry(self.chemistry)
        model = self.setup_simulation.set_electrochemical_model(self.electrochemical_model)
        solver = self.setup_simulation.set_solver(self.solver)

        simulation = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver)

        # Example: Simulating for a specific time range
        time_span = [0, 3700]  # Adjust this according to your needs
        solution = simulation.solve(time_span)

        return solution

if __name__ == "__main__":
    sim = SimulateBattery(chemistry="LFP", electrochemical_model="DFN", solver="CasadiSolver")
    simulation = sim.simulate_battery_base_model()
    
    simulation.plot()

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …

In [120]:
""" ************************************************************************************* """
""" ********************         chemistry + model       ******************************** """
""" ************************************************************************************* """
import pybamm
from pydantic import BaseModel, StrictStr, validator

NMC=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json")
LFP=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json")

class SetupModel(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    # make a validator to validate if the the chemistry is not in the config then cant work
    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}: # update this condition so if not in the battery.config
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")

    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        
    # not sure if this is the best way but for right now it's fine
    def set_bpx_model(self) -> pybamm.ParameterValues:
        print(f"bpx_model: {self.bpx_model}")
        if self.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues("chen2020")
            print("Parameter values for LFP:", parameter_values)  # Debug print
            return parameter_values
        elif self.bpx_model == "NMC":
            # parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
            print("Parameter values for NMC:", parameter_values)  # Debug print
            return parameter_values
        else:
            raise ValueError("Unexpected bpx_model value encountered")


""" ************************************************************************************* """
""" **************************         Solver       ************************************* """
""" ************************************************************************************* """
import pybamm
from pydantic import BaseModel, StrictStr, validator, StrictFloat

# Since the solver func is so diverse, I chose to seperate out the sovler into it's own class
# very basic concept of it however there is many different solvers so may make classes inherit form a generic solver class
# will look at Casadi, JAX, IDAKALU and Base model
# all solvers will have a tolerence in common (atol, rtol)
class DefineSolver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

# want to make this more fault tollerent - have it if not in a list from a solver_config file of available solvers to use
@validator("solver")
def validate_mode(cls, solver):
    if solver not in {"CasadiSolver", "BaseModel", "JAX"}:
        raise ValueError("Invalid mode for the solver, must be Casadi, BaseModel or JAX")
    return solver

""" ************************************************************************************* """
""" ************************         Casdadi       ************************************** """
""" ************************************************************************************* """
# Define casadi solver and inherit from the base class 
class CasadiSolver(DefineSolver):
    mode: StrictStr

    @validator("mode")
    def validate_casadi_mode(cls, mode):
        if mode not in {"safe", "fast"}:
            raise ValueError("Invalid sovler mode, CasadiSolver must be solved with 'safe' or 'fast'")
        return mode

    @validator("solver")
    def validate_solver(cls, value):
        if value != "CasadiSolver":
            raise ValueError("solver must be 'CasadiSolver' for CasadiSolver class")
        return value

    def set_casadi_solver(self):
        solver = pybamm.CasadiSolver(mode=self.mode, atol=self.atol, rtol=self.rtol)
        solver._on_extrapolation = "warn"  
        return solver

""" ************************************************************************************* """
""" ****************************         Tests       ************************************ """
""" ************************************************************************************* """
def test_setup_model():
        try:
            setupModel = SetupModel(
                bpx_model="LFP",
                electrochemical_model="DFN",
            )
            print(f"bpx_model after initialization: {setupModel.bpx_model}")


            parameter_values = setupModel.set_bpx_model()
            print("Parameter values for BPX model:", parameter_values)

            # Test set_casadi_solver
            casadi_solver = CasadiSolver(solver="CasadiSolver", atol=1e-6, rtol=1e-6, mode="safe")
            solver_instance = casadi_solver.set_casadi_solver()
            print("Casadi solver instance:", solver_instance)
        
        except ValueError as e:
            print(e)
""" ************************************************************************************* """
""" ****************************         Main       ************************************* """
""" ************************************************************************************* """
if __name__ == "__main__":
    test_setup_model()

bpx_model after initialization: None
bpx_model: None
Unexpected bpx_model value encountered


In [1]:
import pybamm
from pydantic import BaseModel, Field, validator
from typing import Dict, Literal, Optional

# Constants
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"
NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"

class SetupSimulation(BaseModel):
    chemistry: Literal["LFP", "NMC"]
    electrochemical_model: Literal["DFN", "SPM", "SPMe"] = "DFN"
    solver: Literal["CasadiSolver"] = "CasadiSolver"
    adjusted_parameters: Dict[str, float] = Field(default_factory=dict)

    ADJUSTABLE_PARAMETERS = [
        "Ambient temperature [K]",
        "Initial temperature [K]",
        "Lower voltage cut-off [V]",
        "Upper voltage cut-off [V]",
        "Current function [A]",
    ]

    def set_battery_chemistry(self) -> pybamm.ParameterValues:
        if self.chemistry == "LFP":
            return pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.chemistry == "NMC":
            return pybamm.ParameterValues.create_from_bpx(NMC)

    def set_electrochemical_model(self) -> pybamm.lithium_ion.BaseModel:
        if self.electrochemical_model == "DFN":
            return pybamm.lithium_ion.DFN()
        elif self.electrochemical_model == "SPM":
            return pybamm.lithium_ion.SPM()
        elif self.electrochemical_model == "SPMe":
            return pybamm.lithium_ion.SPMe()

    def set_solver(self) -> pybamm.BaseSolver:
        solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)
        solver._on_extrapolation = "warn"
        return solver

    @validator('adjusted_parameters')
    def validate_adjusted_parameters(cls, v):
        for key in v:
            if key not in cls.ADJUSTABLE_PARAMETERS:
                raise ValueError(f"Invalid parameter: {key}")
        return v

class SimulateBattery(BaseModel):
    setup: SetupSimulation

    def simulate_battery_base_model(self):
        parameter_values = self.setup.set_battery_chemistry()
        model = self.setup.set_electrochemical_model()
        solver = self.setup.set_solver()

        simulation = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver)

        # Example: Simulating for a specific time range
        time_span = [0, 3700]  # Adjust this according to your needs
        solution = simulation.solve(time_span)

        return solution

if __name__ == "__main__":
    setup = SetupSimulation(chemistry="LFP", electrochemical_model="DFN", solver="CasadiSolver")
    sim = SimulateBattery(setup=setup)
    simulation = sim.simulate_battery_base_model()
    
    simulation.plot()

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …

In [86]:
import pybamm
from pydantic import BaseModel, StrictStr, validator, StrictFloat, ValidationError

NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"

class Config_Model(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}:
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")
        return bpx_model
    
    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        return electrochemical_model
        
    def set_bpx_model(self):
        if self.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.bpx_model == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
        else:
            raise ValueError(f"Invalid BPX Model Type: {self.bpx_model}. Use LFP or NMC")
        
        return parameter_values


class Config_Solver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

    @validator("solver")
    def validate_solver(cls, solver):
        if solver != "CasadiSolver":
            raise ValueError("Invalid solver, must be CasadiSolver")
        return solver
    
    def configure_and_get_solver(self):
        if self.solver == "CasadiSolver":
            return self.configure_casadi_solver()
        else:
            raise ValueError(f"Unknown solver type: {self.solver}")

    def configure_casadi_solver(self):
        solver = pybamm.CasadiSolver(mode="safe", atol=self.atol, rtol=self.rtol)
        solver._on_extrapolation = "warn"
        return solver

if __name__ == "__main__":
    setup_model = Config_Model(bpx_model="LFP", electrochemical_model="DFN")
    parameter_values = setup_model.set_bpx_model()

    solver_config = Config_Solver(solver="CasadiSolver", atol=1e-6, rtol=1e-6)
    solver_instance = solver_config.configure_and_get_solver()

    sim = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver_instance)
        
    # Solve and plot
    t_eval = [0, 3700]
    sim.solve(t_eval)
    sim.plot()



The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …

In [15]:
import pybamm
from pydantic import BaseModel, StrictStr, StrictFloat, ValidationError

NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"

class Config_Model(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    @staticmethod
    def create_from_config(bpx_model: str, electrochemical_model: str):
        return Config_Model(bpx_model=bpx_model, electrochemical_model=electrochemical_model)

    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}:
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")
        return bpx_model
    
    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        return electrochemical_model
        
    def set_bpx_model(self):
        if self.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.bpx_model == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
        else:
            raise ValueError(f"Invalid BPX Model Type: {self.bpx_model}. Use LFP or NMC")
        
        return parameter_values
    
    def create_battery_model(self):
        if self.electrochemical_model == "DFN":
            model = pybamm.lithium_ion.DFN()
        elif self.electrochemical_model == "SPM":
            model = pybamm.lithium_ion.SPM()
        elif self.electrochemical_model == "SPMe":
            model = pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Invalid electrochemical model: {self.electrochemical_model}")
        
        return model


class Config_Solver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

    @staticmethod
    def create_from_config(solver: str, atol: float = 1e-6, rtol: float = 1e-6):
        return Config_Solver(solver=solver, atol=atol, rtol=rtol)

    @validator("solver")
    def validate_solver(cls, solver):
        if solver != "CasadiSolver":
            raise ValueError("Invalid solver, must be CasadiSolver")
        return solver
    
    def configure_and_get_solver(self):
        if self.solver == "CasadiSolver":
            return self.configure_casadi_solver()
        else:
            raise ValueError(f"Unknown solver type: {self.solver}")

    def configure_casadi_solver(self):
        solver = pybamm.CasadiSolver(mode="safe", atol=self.atol, rtol=self.rtol)
        solver._on_extrapolation = "warn"
        return solver


class SetupSimulation(BaseModel):
    config_model: Config_Model
    solver_config: Config_Solver

    # THIS WOULD BELONG HERE
    def create_from_config(config_model: Config_Model, solver_config: Config_Solver):
        return SetupSimulation(config_model=config_model, solver_config=solver_config)

    # THIS WOULD BE IT'S OWN CLASS
    def simulate(self, t_eval):
        parameter_values = self.config_model.set_bpx_model()
        solver_instance = self.solver_config.configure_and_get_solver()
        model = self.config_model.create_battery_model()

        sim = pybamm.Simulation(model=model, parameter_values=parameter_values, solver=solver_instance)
        sim.solve(t_eval)
        sim.plot()


if __name__ == "__main__":
    # Define the configuration using nested models
    setup_simulation = SetupSimulation(
        config_model=Config_Model(bpx_model="LFP", electrochemical_model="DFN"),
        solver_config=Config_Solver(solver="CasadiSolver", atol=1e-6, rtol=1e-6)
    )
    
    # Simulate and plot base model over specified time
    t_eval_base_model = [0, 3700]
    setup_simulation.simulate(t_eval_base_model)

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …

In [38]:
import pybamm
from pydantic import BaseModel, StrictStr, StrictFloat, ValidationError

NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"

class Config_Model(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    @staticmethod
    def create_from_config(bpx_model: str, electrochemical_model: str):
        return Config_Model(bpx_model=bpx_model, electrochemical_model=electrochemical_model)

    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}:
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")
        return bpx_model
    
    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        return electrochemical_model
        
    def set_bpx_model(self):
        if self.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.bpx_model == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
        else:
            raise ValueError(f"Invalid BPX Model Type: {self.bpx_model}. Use LFP or NMC")
        
        return parameter_values
    
    def create_battery_model(self):
        if self.electrochemical_model == "DFN":
            model = pybamm.lithium_ion.DFN()
        elif self.electrochemical_model == "SPM":
            model = pybamm.lithium_ion.SPM()
        elif self.electrochemical_model == "SPMe":
            model = pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Invalid electrochemical model: {self.electrochemical_model}")
        
        return model


class Config_Solver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

    @staticmethod
    def create_from_config(solver: str, atol: float = 1e-6, rtol: float = 1e-6):
        return Config_Solver(solver=solver, atol=atol, rtol=rtol)

    @validator("solver")
    def validate_solver(cls, solver):
        if solver != "CasadiSolver":
            raise ValueError("Invalid solver, must be CasadiSolver")
        return solver
    
    def configure_and_get_solver(self):
        if self.solver == "CasadiSolver":
            return self.configure_casadi_solver()
        else:
            raise ValueError(f"Unknown solver type: {self.solver}")

    def configure_casadi_solver(self):
        solver = pybamm.CasadiSolver(mode="safe", atol=self.atol, rtol=self.rtol)
        solver._on_extrapolation = "warn"
        return solver


class SetupSimulation(BaseModel):
    config_model: Config_Model
    solver_config: Config_Solver

    # THIS WOULD BELONG HERE
    def create_from_config(config_model: Config_Model, solver_config: Config_Solver):
        return SetupSimulation(config_model=config_model, solver_config=solver_config)

    # THIS WOULD BE IT'S OWN CLASS
    def simulate(self, t_eval):
        parameter_values = self.config_model.set_bpx_model()
        solver_instance = self.solver_config.configure_and_get_solver()
        model = self.config_model.create_battery_model()

        sim = pybamm.Simulation(model=model, parameter_values=parameter_values, solver=solver_instance)
        sim.solve(t_eval)
        sim.plot()


if __name__ == "__main__":
    # Define the configuration using nested models
    setup_simulation = SetupSimulation(
        config_model=Config_Model(bpx_model="LFP", electrochemical_model="DFN"),
        solver_config=Config_Solver(solver="CasadiSolver", atol=1e-6, rtol=1e-6)
    )
    
    # Simulate and plot base model over specified time
    t_eval_base_model = [0, 3700]
    setup_simulation.simulate(t_eval_base_model)

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …

In [34]:
import pybamm
from pydantic import BaseModel, StrictStr, StrictFloat, ValidationError, validator
from typing import List

NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"

class SetupModel(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    @staticmethod
    def create_from_config(bpx_model: str, electrochemical_model: str):
        return SetupModel(bpx_model=bpx_model, electrochemical_model=electrochemical_model)

    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}:
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")
        return bpx_model
    
    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        return electrochemical_model
    
class SetupSolver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

    @staticmethod
    def create_solver(solver: str, atol: float = 1e-6, rtol: float = 1e-6):
        return SetupSolver(solver=solver, atol=atol, rtol=rtol)
    
    @validator("solver")
    def validate_solver(cls, solver):
        if solver != "CasadiSolver":
            raise ValueError("Invalid solver, must be CasadiSolver")
        return solver

class SimulationRunner:
    def __init__(self, setup_model: SetupModel, setup_solver: SetupSolver):
        self.setup_model = setup_model
        self.setup_solver = setup_solver

    def set_bpx_model(self):
        if self.setup_model.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.setup_model.bpx_model == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
        else:
            raise ValueError(f"Invalid BPX Model Type: {self.setup_model.bpx_model}. Use LFP or NMC")
        
        return parameter_values
    
    def set_electrochemical_model(self):
        if self.setup_model.electrochemical_model == "DFN":
            model = pybamm.lithium_ion.DFN()
        elif self.setup_model.electrochemical_model == "SPM":
            model = pybamm.lithium_ion.SPM()
        elif self.setup_model.electrochemical_model == "SPMe":
            model = pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Invalid electrochemical model: {self.setup_model.electrochemical_model}")
        
        return model

    def configure_solver(self):
        if self.setup_solver.solver == "CasadiSolver":
            solver = pybamm.CasadiSolver(mode="safe", atol=self.setup_solver.atol, rtol=self.setup_solver.rtol)
            solver._on_extrapolation = "warn"
            return solver
        else:
            raise ValueError(f"Unknown solver type: {self.setup_solver.solver}")

    def run_simulation(self, t_eval: List[float]):
        parameter_values = self.set_bpx_model()
        solver_instance = self.configure_solver()
        model = self.set_electrochemical_model()

        sim = pybamm.Simulation(model=model, parameter_values=parameter_values, solver=solver_instance)
        
        try:
            solution = sim.solve(t_eval)
            if not isinstance(solution, pybamm.Solution):
                raise ValueError("Solver did not return a valid solution object.")
            sim.plot(solution)
        except Exception as e:
            print(f"An error occurred during the simulation: {e}")

def main():
    model_config = {"bpx_model": "LFP", "electrochemical_model": "DFN"}
    solver_config = {"solver": "CasadiSolver", "atol": 1e-6, "rtol": 1e-6}
    
    try:
        setup_model = SetupModel.create_from_config(**model_config)
        setup_solver = SetupSolver.create_solver(**solver_config)
        
        runner = SimulationRunner(setup_model=setup_model, setup_solver=setup_solver)
        t_eval = [0, 3700]
        runner.run_simulation(t_eval)
        
    except ValidationError as e:
        print(f"Validation error: {e}")
    except ValueError as e:
        print(f"Value error: {e}")

if __name__ == "__main__":
    main()


The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.


An error occurred during the simulation: object of type 'Solution' has no len()


At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


AttributeError: type object 'ParameterValues' has no attribute 'from_json'

In [64]:
import pybamm
from pydantic import BaseModel, StrictStr, StrictFloat, ValidationError

NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"

class ConfigModel(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    @staticmethod
    def create_from_config(bpx_model: str, electrochemical_model: str):
        return ConfigModel(bpx_model=bpx_model, electrochemical_model=electrochemical_model)

    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}:
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")
        return bpx_model
    
    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        return electrochemical_model
        
    def set_bpx_model(self):
        if self.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.bpx_model == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
        else:
            raise ValueError(f"Invalid BPX Model Type: {self.bpx_model}. Use LFP or NMC")
        
        return parameter_values
    
    def set_electrochemical_model(self):
        if self.electrochemical_model == "DFN":
            model = pybamm.lithium_ion.DFN()
        elif self.electrochemical_model == "SPM":
            model = pybamm.lithium_ion.SPM()
        elif self.electrochemical_model == "SPMe":
            model = pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Invalid electrochemical model: {self.electrochemical_model}")
        
        return model

class ConfigSolver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

    @staticmethod
    def create_from_config(solver: str, atol: float = 1e-6, rtol: float = 1e-6):
        return SetupSolver(solver=solver, atol=atol, rtol=rtol)
    
    @validator("solver")
    def validate_solver(cls, solver):
        if solver != "CasadiSolver":
            raise ValueError("Invalid solver, must be CasadiSolver")
        return solver

    def set_solver(self):
        if solver == "CasadiSolver":
            solver = pybamm.CasadiSolver(mode="safe", atol=self.atol, rtol=self.rtol)
            solver._on_extrapolation = "warn"
        return solver

class BaseSimulation(BaseModel):
    config_model: ConfigModel
    config_solver: ConfigSolver

    def construct_simulation_model(self):
        model = self.config_model.set_electrochemical_model()
        parameter_values = self.config_model.set_bpx_model()
        solver = self.config_solver.set_solver()

        simulation_model = {
            "model": model,
            "parameter_values": parameter_values,
            "solver": solver
        }
        return simulation_model

class TimeEvaluationSimulation(BaseSimulation):
    t_eval: list

    def simulate(self):
        simulation_model = self.construct_simulation_model()

        sim = pybamm.Simulation(model=simulation_model["model"],
                                parameter_values=simulation_model["parameter_values"],
                                solver=simulation_model["solver"])
        
        solution = sim.solve(t_eval)
        sim.plot(solution)


if __name__ == "__main__":
    model_config = {"bpx_model": "LFP", "electrochemical_model": "DFN"}
    config_solver = {"solver": "CasadiSolver", "atol": 1e-6, "rtol": 1e-6}
    t_eval = [0, 3700]

    config_model = ConfigModel.create_from_config(**model_config)
    setup_solver  = ConfigSolver.create_from_config(**config_solver)

    time_eval_simulation = TimeEvaluationSimulation(config_model=config_model,
                                        config_solver=config_solver,
                                        t_eval=t_eval)
    time_eval_simulation.simulate()

UnboundLocalError: cannot access local variable 'solver' where it is not associated with a value

In [70]:
import pybamm
from pydantic import BaseModel, StrictStr, StrictFloat, ValidationError

NMC = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\NMC\AE_gen1_BPX.json"
LFP = r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Models\LFP\lfp_18650_cell_BPX.json"

class ConfigModel(BaseModel):
    bpx_model: StrictStr 
    electrochemical_model: StrictStr

    @staticmethod
    def create_from_config(bpx_model: str, electrochemical_model: str):
        return ConfigModel(bpx_model=bpx_model, electrochemical_model=electrochemical_model)

    @validator("bpx_model")
    def validate_chemistry(cls, bpx_model):
        if bpx_model not in {"LFP", "NMC"}:
            raise ValueError(f"Invalid BPX Model Type: {bpx_model}. Use LFP or NMC")
        return bpx_model
    
    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, electrochemical_model):
        if electrochemical_model not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {electrochemical_model}. Model must be defined as DFN, SPM or SPMe")
        return electrochemical_model
        
    def set_bpx_model(self):
        if self.bpx_model == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
        elif self.bpx_model == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
        else:
            raise ValueError(f"Invalid BPX Model Type: {self.bpx_model}. Use LFP or NMC")
        
        return parameter_values
    
    def set_electrochemical_model(self):
        if self.electrochemical_model == "DFN":
            model = pybamm.lithium_ion.DFN()
        elif self.electrochemical_model == "SPM":
            model = pybamm.lithium_ion.SPM()
        elif self.electrochemical_model == "SPMe":
            model = pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Invalid electrochemical model: {self.electrochemical_model}")
        
        return model

class ConfigSolver(BaseModel):
    solver: StrictStr
    atol: StrictFloat = 1e-6
    rtol: StrictFloat = 1e-6

    @staticmethod
    def create_from_config(solver: str, atol: float = 1e-6, rtol: float = 1e-6):
        return ConfigSolver(solver=solver, atol=atol, rtol=rtol)
    
    @validator("solver")
    def validate_solver(cls, solver):
        if solver != "CasadiSolver":
            raise ValueError("Invalid solver, must be CasadiSolver")
        return solver

    def set_solver(self):
        solver = pybamm.CasadiSolver(mode="safe", atol=self.atol, rtol=self.rtol)
        solver._on_extrapolation = "warn"
        return solver

class BaseSimulation(BaseModel):
    config_model: ConfigModel
    config_solver: ConfigSolver

    def construct_simulation_model(self):
        model = self.config_model.set_electrochemical_model()
        parameter_values = self.config_model.set_bpx_model()
        solver = self.config_solver.set_solver()

        simulation_model = {
            "model": model,
            "parameter_values": parameter_values,
            "solver": solver
        }
        return simulation_model

class TimeEvaluationSimulation(BaseSimulation):
    t_eval: list

    def simulate(self):
        simulation_model = self.construct_simulation_model()

        sim = pybamm.Simulation(model=simulation_model["model"],
                                parameter_values=simulation_model["parameter_values"],
                                solver=simulation_model["solver"])
        
        sim.solve(self.t_eval)
        sim.plot()

class SimulationRunner():
    model_config = {"bpx_model": "LFP", "electrochemical_model": "DFN"}
    config_solver = {"solver": "CasadiSolver", "atol": 1e-6, "rtol": 1e-6}
    t_eval = [0, 3700]

    config_model = ConfigModel.create_from_config(**model_config)
    config_solver = ConfigSolver.create_from_config(**config_solver)

    time_eval_simulation = TimeEvaluationSimulation(config_model=config_model,
                                        config_solver=config_solver,
                                        t_eval=t_eval)
    time_eval_simulation.simulate()

if __name__ == "__main__":
    SimulationRunner()


The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …